<a href="https://colab.research.google.com/github/grifire/tp-initiation-llm-student-version/blob/main/TP_initiation_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/vincentmartin/tp-initiation-llm-student-version/blob/main/TP_initiation_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP d'initiation aux LLM

Dans ce TP, vous allez apprendre les bases de l'IA générative en manipulant et en contrôlant un LLM installé en local.

En sortie de ce module, vous serez capable de :
- Installer et importer les dépendances nécessaires
- Interroger un LLM pour répondre à tout type de question, comme avec chatGPT
- Analyser le fonctionnement d'un LLM
- Utiliser un LLM pour résumer une conversation
- Explorer les techniques de zero-shot, one-shot et few-shot inference

### Instruction à suivre pour exécution sur Google Colab

Aller dans `Execution -> Modifier le type d'exécution` puis sélectionner `T4-GPU` pour exploiter les fonctionnalités GPU.

![Colab GPU](resources/colab_gpu.png "T4-GPU")

## Installation des dépendances

Installons les dépendances nécessaires :
- **transformers** : la bibliothèque permettant de mettre en oeuvre les LLM exploitant le modèle transformers
- **torch** : célèbre bibliothèque de deep learning, sous jacente à transformers

In [ ]:
%pip install -U datasets

%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch \
    torchdata

%pip install \
    transformers

Chargeons les dépendances.

**Remarque : Si l'exécution ressort en erreur ; tenter de recharger les dépendances.**

In [1]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

## Chargement du LLM

Pour interagir avec un LLM, nous allons d'abord devoir le télécharger. Pour cet exemple, nous choissons un modèle simple et "léger" : flan-t5.

Nous chargeons également le **Tokenizer** afin de convertir le texte en tokens et vice-versa.

In [10]:
model_name='google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)


**Exercice** : en vous aidant de la documentation https://huggingface.co/docs/transformers/llm_tutorial :
- Générer et afficher les tokens (ids) de la phrase (encodage)
- Décoder la liste de tokens (ids) et afficher la phrase (décodage)

In [11]:
sentence = "Que peux-tu me dire sur les LLMs ?"

# Encoder la phrase en tokens : suite d'ID ; 1 ID = 1 token
#tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", padding_side="left")
model_inputs = tokenizer([sentence], return_tensors="pt").to(model.device)
# METTRE ICI LE CODE POUR ENCODER UNE PHRASE EN TOKEN ET L'AFFICHER
print(model_inputs["input_ids"])
# METTRE ICI LE CODE POUR DECOER UNE SEQUENCE D'ID EN PHRASE ET L'AFFICHER
decoded_sentence = tokenizer.batch_decode(model_inputs["input_ids"], skip_special_tokens=True)[0]
print("Decoded text:", decoded_sentence)

tensor([[ 7227,     3, 16162,    18,    17,    76,   140,  5794,   244,   110,
           301, 11160,     7,     3,    58,     1]])
Decoded text: Que peux-tu me dire sur les LLMs ?


## Interrogation du LLM

A présent, utilisons notre LLM pour générer du texte.

Notez la syntaxe `User: question? Assistant: "`. Nous utilisons cette syntaxe car le LLM est un modèle qui génère la suite de la phrase et cette syntaxe lui permet de comprendre ce qu'on attend de lui. Ceci à la différence des modèles d'instruction qui génèrent une réponse pour une instruction donnée.

In [12]:
sentence = "User: quelle est la capitale de la france ?Assistant: "

inputs = tokenizer(sentence, return_tensors='pt') # return les tenseurs au format pytorch
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50
    )[0],
    skip_special_tokens=True # Ne pas retourner les tokens <s>, </s>, ...
)

print(output)

Paris


C'est assez basique pour l'instant mais ne vous inquiétez pas, ce n'est que le premier TP ;).

## Résumé de dialogues

Dans cette partie, nous allons utiliser le LLM pour résumer des dialogues.

Tout d'abord, téléchargeons le dataset [DialogSum](https://huggingface.co/datasets/knkarthick/dialogsum) depuis Huggingface

In [13]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)

README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Affichons 2 exemples de dialogues, les exemples numéro 40 et 200.

In [14]:
example_indices = [40, 200]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print('DIALOGUE D ENTREE:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('RESUME HUMAIN:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
DIALOGUE D ENTREE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
RESUME HUMAIN:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Example  

Tentons une première approche pour résumer les dialogues 40 et 200.

In [15]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    inputs = tokenizer(dialogue, return_tensors='pt') # retourner les tenseurs
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50, # max 50 tokens générés
        )[0],
        skip_special_tokens=True # on ne génère pas les tokens spéciaux <, >, ...
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'DIALOGUE D ENTREE::\n{dialogue}')
    print(dash_line)
    print(f'RESUME HUMAIN:\n{summary}')
    print(dash_line)
    print(f'RESUME PAR LLM SANS PROMPT ENGINEERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
DIALOGUE D ENTREE::
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
RESUME HUMAIN:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
RESUME PAR LLM SANS PROMPT ENGINEERING:
Person1: It's ten to nine.

-----------------------------------------

**Exercice** : selon vous est-ce que le résumé est bon ? Pourquoi ?

Le résumé du premier dialogue est complètement hors sujet car il ne fait que répéter un morçaux de phrase du deuxième personnage qui ne donne aucun contexte, en plus de le résumer sur le mauvais personnage.

Le deuxième résumé est correcte mais se trompe de personnage.

## Résumé avec un prompt Instruction

Dans l'exemple ci-dessous, ajoutons une instruction indiquant au LLM ce qu'il doit faire.

### 1. Zero shot inference

Pour amener le modèle à accomplir une tâche, comme résumer un dialogue, vous pouvez transformer ce dialogue en une consigne spécifique. Cela est connu sous le nom d'inférence zéro-shot.

En encadrant le dialogue dans une consigne descriptive, vous pourrez observer les modifications apportées au texte généré.

In [16]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
Summarize the following conversation between two persons to extract the key points of the conversation.

{dialogue}

Summary:
    """

    # Input constructed prompt instead of the dialogue.
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'DIALOGUE D ENTREE:\n{prompt}')
    print(dash_line)
    print(f'RESME HUMAIN:\n{summary}')
    print(dash_line)
    print(f'>>>RESME AVEC ZERO SHOT INFERENCE:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
DIALOGUE D ENTREE:

Summarize the following conversation between two persons to extract the key points of the conversation.

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

Summary:
    
---------------------------------------------------------------------------------------------------
RESME HUMAIN:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
------------------------------------------------------------------------------------------

C'est déjà mieux ! Mais on peut encore faire mieux. Essayons de rajouter un exemple de résumé.

### 2. One Shot Inference

L'inférence one-shot et few-shot consiste à fournir au modèle de langage un ou plusieurs exemples complets de paires consigne-réponse correspondant à votre tâche avant de lui soumettre la consigne réelle que vous souhaitez qu'il accomplisse. Cela s'appelle "l'apprentissage en contexte" (_in context learning_), et cela permet au modèle de comprendre votre tâche spécifique. Pour en savoir plus, vous pouvez consulter [cet article](https://huggingface.co/blog/few-shot-learning-gpt-neo-and-inference-api).

Définissons une fonction qui permet de générer un prompt avec 1 exemple de dialogue et son résumé.

In [17]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']

        # The stop sequence '{summary}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence.
        prompt += f"""
Dialogue:

{dialogue}

Summary:
{summary}


"""

    dialogue = dataset['test'][example_index_to_summarize]['dialogue']

    prompt += f"""
Dialogue:

{dialogue}

Summary:
"""

    return prompt

Construsons le prompt et affichons le.

In [18]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)
# one_shot_prompt is a string
print(one_shot_prompt)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

Summary:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.



Dialogue:

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more 

Lançons l'inférence sur un dialogue, qui doit bien entendu être différent de celui utilisé pour réaliser l'exemple.

In [19]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'RESME HUMAIN:\n{summary}\n')
print(dash_line)
print(f'>>>RESME LLM AVEC ONE SHOT INFERENCE:\n{output}')

---------------------------------------------------------------------------------------------------
RESME HUMAIN:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
>>>RESME LLM AVEC ONE SHOT INFERENCE:
#Person1#: You could add a painting program to your software. #Person2#: That would be a bonus. #Person1#: You might also want to upgrade your hardware because it is pretty outdated now


Voilà qui est encore mieux !!

### 3. Few shot inference

Essayons à présent de fournir 3 exemples de paires (dialogue, résumé). C'est ce que l'on appelle le **few shot inference**.

In [29]:
example_indices_full = [200, 300, 400] # exemples à fournir
example_index_to_summarize = 201 # dialogue à résumer

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)


Dialogue:

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Summary:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.



Dialogue:

#Person1#: I cannot imagine if Trump were to be our President again.
#Person2#: I am proud to say that he is

In [30]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'RESUME HUMAIN:\n{summary}\n')
print(dash_line)
print(f'>>>RESUME LLM AVEC FEW SHOT INFERENCE:\n{output}')

---------------------------------------------------------------------------------------------------
RESUME HUMAIN:
#Person1# is driving #Person2# to an inn. They talk about their careers, ages, and where they was born.

---------------------------------------------------------------------------------------------------
>>>RESUME LLM AVEC FEW SHOT INFERENCE:
#Person1 is flying to China from Mexico. #Person2 is from Mexico. #Person1 is from 40 years old. #Person2 is from Mexico.


**Exercice** : modifier les exemples fournis en entrée et indiquer ce que vous constater en commentaire dans une section markdown.

En fonction des exemples, la qualité des résumé varie. Parmis les erreur que je retrouve souvant il y a l'invesion entre le pays de départ et le pays d'arrivé, et l'inversion des personnage. Mise à par cela, le résumé reste  cohérent. Quand on lui passe en exemple le teste, le résumé est très bon.

## Influence des paramètres du LLM

Nous allons maintenant faire varier plusieurs paramètres du LLM :
- température
- top_k
- top_p
- sampling

Pour cela aidez-vous de la documentation :
- https://huggingface.co/docs/transformers/generation_strategies
- https://huggingface.co/docs/transformers/main_classes/text_generation

**Exercice** : créer une fonction qui prend les 4 paramètres ci-dessous et le paramètres _few_shot_prompt_ défini précédemment et qui retourne le résultat de la génération.

In [ ]:
def generate_summary(temperature, top_k, top_p, sampling, prompt):
    # VOTRE CODE ICI
    pass # à enlever

**Exercice** : expliquer en 1 ou 2 lignes l'influence de chacun des 4 paramètres (dans une section markdown).

### Mini projet : Système de Questions-Réponses avec Classement de Qualité

**Objectif** : Créer un système qui utilise un LLM pour répondre à des questions sur un domaine spécifique et évaluer la qualité des réponses.

**Durée estimée** : 3-4 heures

**Description du projet** :
1. **Créer un dataset de questions-réponses** : Définir 10-15 questions sur un domaine de votre choix (technologie, cinéma, histoire, etc.) avec leurs réponses de référence
2. **Tester différentes approches de prompt** :
    - Zero-shot
    - One-shot
    - Few-shot
3. **Comparer les paramètres de génération** : Tester différentes combinaisons de température, top_k, top_p pour voir leur impact
4. **Évaluation automatique** : Créer une fonction qui compare la réponse du LLM avec la réponse de référence (vous pouvez utiliser des métriques simples comme la longueur, les mots-clés communs, etc.)
5. **Visualisation des résultats** : Afficher un tableau comparatif des performances selon les différentes approches

**Livrables attendus** :
- Code documenté avec des commentaires
- Analyse des résultats en markdown (quelle approche fonctionne le mieux ?)
- Au moins 2 visualisations (graphiques ou tableaux)

## Pour aller plus loin : langchain

Faire le short course https://www.deeplearning.ai/short-courses/langchain-for-llm-application-development/

A partir des compétences fraîchement acquises, réaliser une chaîne de traitement qui
- Demande à l'utilisateur de charger un document PDF ou le charger depuis une URL
- Permet à l'utiisateur de poser des questions sur ce document
- Mémorise les conversations pour répondre aux questions

Votre application devra utiliser des templates de prompts.

- Suggestion de LLM à utiliser : https://huggingface.co/Qwen/Qwen2.5-3B-Instruct.